In [4]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets


In [5]:
# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# Hyperparameters for fully connected NN.

input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epoch = 2

In [16]:
# Create a RNN
class RNN(nn.Module):
    def __init__(self , input_size , hidden_size , num_layers , num_classes):
        super(RNN , self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.RNN = nn.RNN(input_size , hidden_size , num_layers , batch_first = True)
        self.fc = nn.Linear(hidden_size * sequence_length , num_classes)
    
    def forward(self , x):
        h0 = torch.zeros(self.num_layers , x.size(0) , self.hidden_size).to(device)

        # forward propogation
        out, _ = self.RNN(x , h0)
        out = out.reshape(out.shape[0] , -1)
        out = self.fc(out)
        return out


In [17]:
# Load Data
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset , batch_size=batch_size , shuffle=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset , batch_size=batch_size , shuffle=True)

In [18]:
# initialize the network
model = RNN(input_size , hidden_size , num_layers , num_classes).to(device)

In [19]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters() , lr=learning_rate)

In [20]:
# Train the netwrork
for epoch in range(num_epoch):
    for batch_idx , (data , target) in enumerate(train_loader):
        # get data to cuda if possible
        data = data.to(device).squeeze(1)
        target = target.to(device)

        #forward
        scores = model(data)
        loss = criterion(scores , target)

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step()

In [25]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    # Toggle model back to train
    model.train()
    return num_correct / num_samples * 100

In [27]:
check_accuracy(train_loader , model)
check_accuracy(test_loader , model)

tensor(0.9628, device='cuda:0')